In [ ]:
def VaR(a, alpha=0.05):

    # 如果是连续型随机变量，生成样本数据
    if hasattr(a, 'rvs'):
        a = a.rvs(size=10000)  # 生成 10000 个样本 , This line is added to generate samples from the distribution

    # 对数据排序
    x = np.sort(a)

    # 计算上界和下界索引
    n = len(a)
    nup = int(np.ceil(n * alpha)) - 1  # Python 索引从 0 开始
    ndn = int(np.floor(n * alpha)) - 1

    # 计算 VaR
    v = 0.5 * (x[nup] + x[ndn])
    return -v

In [ ]:
def ES(x, alpha=0.05):
    """
    计算 Value at Risk (VaR) 和 Expected Shortfall (ES)。

    参数:
        x (numpy.ndarray): 数据数组。
        alpha (float): 置信水平，默认为 0.05。

    返回:
        tuple: VaR 和 ES 的负值。
    """
    # 对数据排序
    xs = np.sort(x)

    n = alpha * len(xs)
    iup = int(np.ceil(n)) - 1  # Python 索引从 0 开始
    idn = int(np.floor(n)) - 1
    ES = np.mean(xs[:idn + 1])  # 包含到第 idn 索引的数据

    return -ES

In [ ]:
def aggregate_risk(values, group_by_cols, alpha=0.05):
    grouped = values.groupby(group_by_cols)
    results = []
    for name, group in grouped:
        pnl_sorted = np.sort(group["pnl"])
        VaR = np.percentile(pnl_sorted, alpha * 100)
        ES = np.mean(pnl_sorted[pnl_sorted <= VaR])
        total_value = group["currentValue"].iloc[0]
        results.append({
            "Stock": name,
            "VaR95": -VaR,
            "ES95": -ES,
            "VaR95_Pct": -VaR / total_value,
            "ES95_Pct": -ES / total_value
        })
    total_current_value = values["currentValue"].mean() * 2
    portfolio_pnl = values.groupby("iteration")["pnl"].sum()
    portfolio_sorted_pnl = np.sort(portfolio_pnl.values)
    portfolio_var95 = -np.percentile(portfolio_sorted_pnl, 5)
    portfolio_es95 = -portfolio_sorted_pnl[portfolio_sorted_pnl <= (-portfolio_var95)].mean()
    results.append({
        "Stock": "Total",
        "VaR95": portfolio_var95,
        "ES95": portfolio_es95,
        "VaR95_Pct": portfolio_var95 / total_current_value,
        "ES95_Pct": portfolio_es95 / total_current_value
    })

    return pd.DataFrame(results)

In [ ]:
# 计算 Spearman 相关性
def spearman_corr(u):
    return pd.DataFrame(u).corr(method="spearman").to_numpy()